In [1]:
import pandas as pd
import numpy as np
from src import soporte as sp

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Fase 1: Exploración y Limpieza
## 1. Exploración Inicial:

In [4]:
df_flight = pd.read_csv("data/Customer Flight Activity.csv")
df_flight.head(2)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0


In [5]:
df_loyalty = pd.read_csv("data/Customer Loyalty History.csv")
df_loyalty.head(2)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN


In [8]:
print(f"El csv de Actividad de Vuelos de los Clientes tiene {df_flight.shape[0]} filas y {df_flight.shape[1]} columnas ")
print("--------------")
print(f"El csv de Historial de los Clientes tiene {df_loyalty.shape[0]} filas y {df_loyalty.shape[1]} columnas ")

El csv de Actividad de Vuelos de los Clientes tiene 405624 filas y 10 columnas 
--------------
El csv de Historial de los Clientes tiene 16737 filas y 16 columnas 


In [ ]:
#al hacer la exploración a la tabla de historial de los clientes, veo que hay valores negativos en la columna de salarios

#verifico que solo es un salario con un value_counts()

# busco el perfil del cliente con ese error en el salario

# filtro por su nivel de estudios

# miro los valores estadisticos de los salarios dentro de eso nivel de educacion

# es un valor lógico si quitamos el negativo

#igualmente compruebo el salario medio por provincias, para ver si se corresponde

#y veo que tanto la media como la mediana (puesto que ese valor negativo puede alterar la media) es de las mas bajas del pais

In [9]:
#El salario negativo lo cambio a un valor absoluto, así deja de ser negativo

df_loyalty["Salary"] = abs(df_loyalty["Salary"])

In [ ]:
# NULOS EN SALARIO

# Pruebo a imputar con Imputer Iteractive,  KNN y la media de la media entre los nivel de estudios "High School or Below" y "Bachelor" 
# y  comparo resultados y  son bastante exactos.

# los cambio por la media de la media entre los salarios de los clientes con nivel de estudios "High School or Below" y "Bachelor"

In [10]:
# Filtrar los salarios de los grupos "High School or Below" y "Bachelor"
salarios_sup_inf = df_loyalty[df_loyalty["Education"].isin(["High School or Below", "Bachelor"])]["Salary"]

# Calcular el salario promedio de esos grupos
salario_promedio = salarios_sup_inf.mean()

# Imputar el salario promedio en los registros con nivel de estudios "College" y salario nulo
df_loyalty["Salary"] = df_loyalty["Salary"].fillna(salario_promedio)

In [23]:
salario_promedio

np.float64(71786.84232033402)

In [ ]:
# Al hacer la exploración de la tabla de actividad de vuelos de los clientes veo que aparecen registros duplicados

# Me creo una variable con los duplicados para visualizarlos

# Cojo un número de cliente y lo veo al detalle y me doy cuenta de que lo que están duplicados son los meses, y que hay registros diferentes 
# aunque coincidan mes y año

In [11]:
# agrupo por loyalty number, año y mes, y que me sume el resto de valores para solucionar problema de registros duplicados (meses)
df_flight_agrupado = df_flight.groupby(["Loyalty Number", "Year", "Month"], as_index=False).agg({'Flights Booked': 'sum', 
                                                                                                 'Flights with Companions': 'sum', 
                                                                                                 'Total Flights': 'sum', 
                                                                                                 'Distance': 'sum', 
                                                                                                 'Points Accumulated': 'sum', 
                                                                                                 'Points Redeemed': 'sum', 
                                                                                                 'Dollar Cost Points Redeemed': 'sum'})

In [12]:
# Convierto los valores de  Cancellation Year y Cancelation month  integer y conservo los nulos
sp.convertir_int(df_loyalty, "Cancellation Year")

In [9]:
sp.convertir_int(df_loyalty, "Cancellation Month")

In [13]:
#Uno los dos dataframes con merge por la columna en comun loyalty number
df_union = df_flight_agrupado.merge(df_loyalty, on = "Loyalty Number")

In [16]:
df_union.sample(6)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
148291,433020,2018,8,15,2,17,4284,428.0,0,0,Canada,British Columbia,West Vancouver,V6V 8Z3,Female,Bachelor,60897.000000,Married,Star,2712.83,2018 Promotion,2018,3,2018,11.0
226758,607480,2017,7,0,0,0,0,0.0,0,0,Canada,British Columbia,Vancouver,V6E 3D9,Female,College,71660.562672,Single,Nova,2781.59,Standard,2016,12,<NA>,NaN
136273,407468,2017,2,0,0,0,0,0.0,0,0,Canada,Manitoba,Winnipeg,R2C 0M5,Female,Bachelor,60297.000000,Married,Nova,2853.51,Standard,2013,6,<NA>,NaN
322562,821515,2017,3,0,0,0,0,0.0,0,0,Canada,British Columbia,Dawson Creek,U5I 4F1,Female,Bachelor,60805.000000,Married,Star,4105.58,2018 Promotion,2018,3,<NA>,NaN
120069,370300,2018,10,4,4,8,2256,225.0,588,48,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,96550.000000,Married,Star,12280.77,Standard,2013,1,<NA>,NaN
157943,454665,2018,12,0,0,0,0,0.0,0,0,Canada,Newfoundland,St. John's,A1C 6H9,Female,Bachelor,60148.000000,Divorced,Nova,8203.22,Standard,2018,12,<NA>,NaN


In [17]:
sp.exploracion(df_union)

El DataFrame tiene 401688 filas y 25 columnas.
    Tiene 0 datos duplicados, lo que supone un porcentaje de 0.0% de los datos.

    Hay 2 columnas con datos nulos, y son: 
    ['Cancellation Year', 'Cancellation Month']

    y sin nulos hay 23 columnas y son: 
    ['Loyalty Number', 'Year', 'Month', 'Flights Booked', 'Flights with Companions', 'Total Flights', 'Distance', 'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed', 'Country', 'Province', 'City', 'Postal Code', 'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card', 'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month']


    A continuación tienes un detalle sobre los datos nulos y los tipos y número de datos:


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Loyalty Number,0.0%,100.0%,int64,16737
Year,0.0%,100.0%,int64,2
Month,0.0%,100.0%,int64,12
Flights Booked,0.0%,100.0%,int64,40
Flights with Companions,0.0%,100.0%,int64,21


Principales estadísticos de las columnas categóricas:


,count,unique,top,freq
Country,401688,1,Canada,401688
Province,401688,11,Ontario,129696
City,401688,29,Toronto,80424
Postal Code,401688,55,V6E 3D9,21864
Gender,401688,2,Female,201840
Education,401688,5,Bachelor,251400
Marital Status,401688,3,Married,233640
Loyalty Card,401688,3,Star,183288
Enrollment Type,401688,2,Standard,378384


Principales estadísticos de las columnas numéricas:


,count,mean,std,min,25%,50%,75%,max
Loyalty Number,401688.0,549735.880445,258904.719886,100018.0,326603.0,550434.0,772019.0,999986.0
Year,401688.0,2017.5,0.500001,2017.0,2017.0,2017.5,2018.0,2018.0
Month,401688.0,6.5,3.452057,1.0,3.75,6.5,9.25,12.0
Flights Booked,401688.0,4.155374,5.269271,0.0,0.0,1.0,8.0,39.0
Flights with Companions,401688.0,1.041916,2.090835,0.0,0.0,0.0,1.0,20.0
Total Flights,401688.0,5.19729,6.576952,0.0,0.0,1.0,10.0,57.0
Distance,401688.0,1220.725451,1446.440549,0.0,0.0,524.0,2352.0,11244.0
Points Accumulated,401688.0,124.904743,147.982107,0.0,0.0,53.0,240.0,1216.5
Points Redeemed,401688.0,30.99766,126.104987,0.0,0.0,0.0,0.0,996.0
Dollar Cost Points Redeemed,401688.0,2.508848,10.20009,0.0,0.0,0.0,0.0,80.0


,% nulos,% no_nulos,tipo_dato,num_valores_unicos
Loyalty Number,0.0%,100.0%,int64,16737
Year,0.0%,100.0%,int64,2
Month,0.0%,100.0%,int64,12
Flights Booked,0.0%,100.0%,int64,40
Flights with Companions,0.0%,100.0%,int64,21
Total Flights,0.0%,100.0%,int64,52
Distance,0.0%,100.0%,int64,5169
Points Accumulated,0.0%,100.0%,float64,1736
Points Redeemed,0.0%,100.0%,int64,591
Dollar Cost Points Redeemed,0.0%,100.0%,int64,52


In [21]:
#compruebo que no haya incoherencia entre el año de cancelación y los vuelos realizados

df_union[(df_union["Year"] > df_union["Cancellation Year"]) & (df_union["Flights Booked"] > 0)]

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month


In [22]:
#guardar CSV

df_union.to_csv("data/union_tablas.csv")